## Importing Packages

In [8]:
import ibov_arb
import pandas as pd
import datetime
import time
from IPython.display import clear_output

## Basic Settings

Connecting to MT5

In [ ]:
LOGIN = ""
SENHA = ""
SERVIDOR = ""

ibov_arb.mt5_connect(login=LOGIN, password=SENHA, server=SERVIDOR)

Extracting IBOVESPA data

In [4]:
composic_ibov = ibov_arb.comp_ibov_hoje()[0]
redutor = ibov_arb.comp_ibov_hoje()[1]
ibov_arb.open_ibov_tickers(composic_ibov)

Dates used for DI interpolation and SPOT carrying

In [5]:
nat_hol = pd.read_excel("feriados_nacionais_anbima.xlsx")

today_date = datetime.date.today()
expiry_date = ibov_arb.find_expiry_day(holidays=nat_hol)

first_B_N = ibov_arb.first_b_day_nextM(date=today_date, holidays=nat_hol)
first_B_N_next = ibov_arb.first_b_day_nextM(date=expiry_date, holidays=nat_hol)

first_leg = ibov_arb.business_days_bet(start=first_B_N, end=expiry_date, holidays=nat_hol) 
second_leg = ibov_arb.business_days_bet(start=expiry_date, end=first_B_N_next, holidays=nat_hol) 
total_days = first_leg + second_leg

dist_expiry = ibov_arb.business_days_bet(start=today_date, end=expiry_date, holidays=nat_hol)

Final Parameters

In [6]:
DI1_ticker = "DI1Z23"
DI2_ticker = "DI1F24"
current_ibov_fut = "WINZ23"

for i in [DI1_ticker, DI2_ticker, current_ibov_fut]:
    ibov_arb.OpenTicker(i)

## Main Execution

In [ ]:
while True:
    risk_free_factor = ibov_arb.get_interp_DI_FACTOR(
        DI1_ticker=DI1_ticker, 
        DI2_ticker=DI2_ticker, 
        first_leg=first_leg, 
        second_leg=second_leg,
        dist_expiry=dist_expiry, 
        method="normal"
    )

    ibov_current = ibov_arb.price_IBOV(comp_ibov=composic_ibov, redutor=redutor)
    future_ibov_mkt = ibov_arb.get_price(current_ibov_fut)
    predicted_price = round(ibov_current * risk_free_factor, 2)

    clear_output(wait=True)
    print(f"Spot IBOV: {ibov_current}")
    print(f"Predicted Price: {predicted_price}")
    print(f"Market Future Price: {future_ibov_mkt}\n")
    print(f"Current Diff % -> {round(((future_ibov_mkt/predicted_price) - 1)*100, 4)}%")
    print(f"Current Diff Points -> {round((future_ibov_mkt-predicted_price), 2)}")
    time.sleep(0.5)